# 数据准备

## 数据导入

In [29]:
from ngboost import NGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import pandas as pd
data = pd.read_csv('/Users/apple/Documents/ML_Project/ML - 2.1/Data/相近8个地点2012年数据/20739-2012.csv',
                  skiprows=[0,1,2])

data.describe()

,Year,Month,Day,Hour,Minute,power (MW),wind direction at 100m (deg),wind speed at 100m (m/s),air temperature at 2m (K),surface air pressure (Pa),density at hub height (kg/m^3)
count,105408.0,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000,105408.000000
mean,2012.0,6.513661,15.756831,11.500000,27.500000,4.125135,193.870912,7.899902,287.464005,84606.872893,1.011686
std,0.0,3.451250,8.811571,6.922219,17.260345,3.694119,88.377513,4.100500,9.776362,468.523564,0.033038
min,2012.0,1.000000,1.000000,0.000000,0.000000,0.000000,0.022000,0.039000,264.279000,83157.784000,0.939000
25%,2012.0,4.000000,8.000000,5.750000,13.750000,0.647000,122.959750,4.797000,279.383000,84290.328000,0.986000
50%,2012.0,7.000000,16.000000,11.500000,27.500000,2.985500,212.370500,7.562000,287.852000,84634.568000,1.008000
75%,2012.0,10.000000,23.000000,17.250000,41.250000,7.813000,267.541000,10.546250,295.039000,84930.296000,1.036000
max,2012.0,12.000000,31.000000,23.000000,55.000000,10.000000,359.989000,33.057000,309.620000,86287.160000,1.102000


## 数据转换

In [28]:
X = data.drop('power (MW)', axis=1)
Y = data['power (MW)']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=520)

from sklearn.preprocessing import MinMaxScaler
X_Scaler = MinMaxScaler()
X_train = X_Scaler.fit_transform(X_train)
X_test = X_Scaler.transform(X_test)

Y_Scaler = MinMaxScaler()
Y_train = Y_Scaler.fit_transform(Y_train.values.reshape(-1,1)).reshape(len(Y_train),)
Y_test = Y_Scaler.transform(Y_test.values.reshape(-1,1)).reshape(len(Y_test),)

# 模型预测

In [34]:
from ngboost import NGBRegressor
def model_test(Base, X_train, X_test, Y_train, Y_test, 
               n_estimators=500, verbose_eval=100):
    ngb = NGBRegressor(Base=Base, n_estimators=n_estimators,verbose_eval=verbose_eval)
    print(ngb,'\n')
    ngb.fit(X_train, Y_train)

    Y_preds = ngb.predict(X_test)
    Y_dists = ngb.pred_dist(X_test) # return norm method: mean std

    # test Mean Squared Error
    test_MSE = mean_squared_error(Y_preds, Y_test)
    print('\nTest MSE', test_MSE)

    # test Negative Log Likelihood
    test_NLL = -Y_dists.logpdf(Y_test).mean()
    print('Test NLL', test_NLL)

## default_linear_learner

In [31]:
from ngboost.learners import default_linear_learner
model_test(Base=default_linear_learner,
           X_train=X_train, X_test=X_test,
           Y_train=Y_train, Y_test=Y_test,
          n_estimators=500, verbose_eval=100)

NGBRegressor(Base=<function default_linear_learner at 0x1a1ea9b488>,
             Dist=<class 'ngboost.distns.normal.Normal'>,
             Score=<class 'ngboost.scores.MLE'>, learning_rate=0.01,
             minibatch_frac=1.0, n_estimators=500, natural_gradient=True,
             tol=0.0001, verbose=True, verbose_eval=100) 

[iter 0] loss=0.4233 val_loss=0.0000 scale=1.0000 norm=0.5075
[iter 100] loss=-0.1296 val_loss=0.0000 scale=1.0000 norm=0.3964
[iter 200] loss=-0.4713 val_loss=0.0000 scale=1.0000 norm=0.3808
[iter 300] loss=-0.6817 val_loss=0.0000 scale=1.0000 norm=0.3446
[iter 400] loss=-0.7670 val_loss=0.0000 scale=1.0000 norm=0.3483

Test MSE 0.015955682176653666
Test NLL -0.7861904540301601


## default_tree_learner

In [32]:
from ngboost.learners import default_tree_learner
model_test(Base=default_tree_learner,
           X_train=X_train, X_test=X_test,
           Y_train=Y_train, Y_test=Y_test,
          n_estimators=500, verbose_eval=100)

NGBRegressor(Base=<function default_linear_learner at 0x1a1ea9b488>,
             Dist=<class 'ngboost.distns.normal.Normal'>,
             Score=<class 'ngboost.scores.MLE'>, learning_rate=0.01,
             minibatch_frac=1.0, n_estimators=500, natural_gradient=True,
             tol=0.0001, verbose=True, verbose_eval=100) 

[iter 0] loss=0.4233 val_loss=0.0000 scale=1.0000 norm=0.5075
[iter 100] loss=-0.1296 val_loss=0.0000 scale=1.0000 norm=0.3964
[iter 200] loss=-0.4713 val_loss=0.0000 scale=1.0000 norm=0.3808
[iter 300] loss=-0.6817 val_loss=0.0000 scale=1.0000 norm=0.3446
[iter 400] loss=-0.7670 val_loss=0.0000 scale=1.0000 norm=0.3483

Test MSE 0.015955682176653666
Test NLL -0.7861904540301601


## esn_ridge_learner

In [ ]:
from ngboost.learners import esn_ridge_learner
model_test(Base=esn_ridge_learner,
           X_train=X_train[:8000], X_test=X_test,
           Y_train=Y_train[:8000], Y_test=Y_test,
          n_estimators=500, verbose_eval=10)

NGBRegressor(Base=<function esn_ridge_learner at 0x1a1fceb7b8>,
             Dist=<class 'ngboost.distns.normal.Normal'>,
             Score=<class 'ngboost.scores.MLE'>, learning_rate=0.01,
             minibatch_frac=1.0, n_estimators=500, natural_gradient=True,
             tol=0.0001, verbose=True, verbose_eval=10) 

[iter 0] loss=0.4216 val_loss=0.0000 scale=1.0000 norm=0.5063
[iter 10] loss=0.3360 val_loss=0.0000 scale=1.0000 norm=0.4536
[iter 20] loss=0.2650 val_loss=0.0000 scale=1.0000 norm=0.4223
[iter 30] loss=0.2032 val_loss=0.0000 scale=1.0000 norm=0.4049
[iter 40] loss=0.1474 val_loss=0.0000 scale=1.0000 norm=0.3961
[iter 50] loss=0.0954 val_loss=0.0000 scale=1.0000 norm=0.3923
[iter 60] loss=0.0469 val_loss=0.0000 scale=1.0000 norm=0.3913
[iter 70] loss=0.0007 val_loss=0.0000 scale=1.0000 norm=0.3919
[iter 80] loss=-0.0436 val_loss=0.0000 scale=1.0000 norm=0.3930
[iter 90] loss=-0.0863 val_loss=0.0000 scale=1.0000 norm=0.3944
[iter 100] loss=-0.1276 val_loss=0.0000 scale=